In [1]:

import requests
import pandas as pd
import joblib
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def fetch_weather_data(api_key, city):
    # Use the city name in quotes
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data['main']['humidity'], data['main']['temp']
    else:
        raise Exception(f"Error fetching weather data: {response.status_code}")

def generate_stress_report(humidity, temperature, step_count):
    # Load the saved model
    model = joblib.load("stress_detection_model.joblib")
    
    # Prepare the input data
    input_data = pd.DataFrame([[humidity, temperature, step_count]], columns=['Humidity', 'Temperature', 'Step_count'])
    
    # Make prediction
    prediction = model.predict(input_data)

    # Create a PDF report
    pdf_file_path = "stress_report.pdf"
    c = canvas.Canvas(pdf_file_path, pagesize=letter)
    c.drawString(100, 750, "Stress Report")
    c.drawString(100, 730, f"Humidity: {humidity}%")
    c.drawString(100, 710, f"Temperature: {temperature}°C")
    c.drawString(100, 690, f"Step Count: {step_count}")
    c.drawString(100, 670, f"Predicted Stress Level: {prediction[0]}")
    c.save()

    return pdf_file_path

# Example usage
api_key = "9433c011de3886e1014d3b25f5ba5f1c"  # Replace with your OpenWeatherMap API key
city = "Chennai"  # Use the city name in quotes
step_count = 123  # Replace with real step count from Apple Watch integration

# Fetch weather data
try:
    humidity, temperature = fetch_weather_data(api_key, city)
    report_path = generate_stress_report(humidity, temperature, step_count)
    print(f"Report generated: {report_path}")
except Exception as e:
    print(e)


Report generated: stress_report.pdf
